### Read exclude.json file

In [31]:
import pandas as pd
import json

# Load the JSON file into a DataFrame
with open("/pscratch/sd/p/pakmasha/ENIGMA_unzip/Rome_SLF/exclude.json", "r") as file:
    data = json.load(file)

df = pd.DataFrame(data)

In [32]:
df.head()

,sub,type,rating,task
0,sub-AOCD001,skull_strip_report,good,NaN
1,sub-AOCD001,t1_norm_rpt,good,NaN
2,sub-AOCD002,skull_strip_report,good,NaN
3,sub-AOCD002,t1_norm_rpt,good,NaN
4,sub-AOCD003,skull_strip_report,uncertain,NaN


In [33]:
df["sub"].nunique()

17

In [34]:
df["rating"].value_counts()

rating
good         78
uncertain    12
bad          12
Name: count, dtype: int64

### Uncertain ratings

In [35]:
uncertain_ratings = df[df["rating"] == "uncertain"]
uncertain_ratings_sorted = uncertain_ratings.sort_values(by="sub")
print(uncertain_ratings_sorted["sub"].nunique())
#uncertain_ratings_sorted

8


In [36]:
# Count unique 'sub' values for each 'type'
unique_sub_counts = uncertain_ratings.groupby('type')['sub'].nunique()

# Display the result
print(unique_sub_counts)

type
bold_conf             2
ica_aroma             5
skull_strip_report    1
tsnr_rpt              4
Name: sub, dtype: int64


In [37]:
#uncertain_ratings_sorted[150:]

### Bad ratings

In [38]:
bad_ratings = df[df["rating"] == "bad"]
bad_ratings_sorted = bad_ratings.sort_values(by="sub")
print(bad_ratings_sorted["sub"].nunique())
bad_ratings_sorted

4


,sub,type,rating,task
8,sub-AOCD001,ica_aroma,bad,rest
36,sub-AOCD001,epi_norm_rpt,bad,rest
37,sub-AOCD001,tsnr_rpt,bad,rest
38,sub-AOCD001,bold_conf,bad,rest
14,sub-AOCD003,ica_aroma,bad,rest
43,sub-AOCD003,tsnr_rpt,bad,rest
44,sub-AOCD003,bold_conf,bad,rest
26,sub-AOCD008,ica_aroma,bad,rest
55,sub-AOCD008,epi_norm_rpt,bad,rest
56,sub-AOCD008,tsnr_rpt,bad,rest


In [39]:
# Count unique 'sub' values for each 'type'
unique_sub_counts = bad_ratings.groupby('type')['sub'].nunique()

# Display the result
print(unique_sub_counts)

type
bold_conf       3
epi_norm_rpt    2
ica_aroma       3
tsnr_rpt        4
Name: sub, dtype: int64


### None ratings

In [40]:
none_ratings = df[df["rating"] == "none"]
none_ratings_sorted = none_ratings.sort_values(by="sub")
#print(none_ratings_sorted["sub"].nunique())
none_ratings_sorted

,sub,type,rating,task


### Get the list of subjects with bad ratings for QC.json file

In [41]:
unformatted = set(bad_ratings_sorted["sub"])
unformatted

{'sub-AOCD001', 'sub-AOCD003', 'sub-AOCD008', 'sub-AOCD009'}

### Read the meta-data file

In [42]:
import pandas as pd

# Define the file path
file_path = "/global/homes/p/pakmasha/ENIGMA-OCD results/QC/Formatted meta-data.xlsx"

# Load the Excel file into a pandas DataFrame
meta_data = pd.read_excel(file_path)

# Display the first few rows to verify the data
print(meta_data.head())

          Sample Subject ID Formatted ID  Unique ID  OCD  Age of onset  \
0  Amsterdam_AMC        101      sub-101        NaN  2.0           NaN   
1  Amsterdam_AMC        103      sub-103        NaN  2.0           NaN   
2  Amsterdam_AMC        104      sub-104        NaN  2.0           NaN   
3  Amsterdam_AMC        105      sub-105        NaN  2.0           NaN   
4  Amsterdam_AMC        106      sub-106        NaN  2.0           NaN   

   Medication  Y-BOCS   Age  Age range  ...  Education  Depression current  \
0         1.0     0.0  49.0        3.0  ...       15.0                 0.0   
1         1.0     2.0  52.0        3.0  ...       15.0                 0.0   
2         1.0     0.0  31.0        3.0  ...       18.0                 0.0   
3         1.0     0.0  24.0        3.0  ...       17.0                 0.0   
4         1.0     0.0  25.0        3.0  ...       18.0                 0.0   

   Depression lifetime  Anxiety current  Anxiety lifetime Agr_Check  Clean  \
0       

### Convert subject IDs with bad ratings to the "Formatted ID"

In [43]:
# Define the specific value for the Sample column
target_sample = "Rome_SLF"  # Replace with the actual value

# Filter the meta_data DataFrame to include only rows with the target Sample value
filtered_meta_data = meta_data[meta_data['Sample'] == target_sample]

# Initialize the formatted list
formatted = []

# Iterate through unformatted IDs and find matching Unique IDs in the filtered DataFrame
for subject_id in unformatted:
    # Check if the subject_id exists in the filtered_meta_data's "Subject ID" column
    match = filtered_meta_data.loc[filtered_meta_data['Subject ID'] == subject_id, 'Formatted ID']
    if not match.empty:
        formatted.append(match.values[0])  # Add the matching Unique ID to the formatted list
    else:
        print(f"subject {subject_id} is not matched")

# Print or use the formatted list
print("Formatted IDs:", formatted)

subject sub-AOCD008 is not matched
subject sub-AOCD001 is not matched
subject sub-AOCD003 is not matched
Formatted IDs: ['sub-subAOCD009']
